In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'numpy'

# **Crushes in Colorado Exploratory data analysis**

Introduction:
* Colorado Springs is a city in Colorado state in the Western United States.
* CSPD is the police of Colorado Springs.
* The dataset was taken from the CSPD.

Tasks:
* Clean the data
* Analyze the dataset
* Present appropriate graphs

## <h2>Preparation</h2>


Import the libraries:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'notebook'


Reading the csv file:

In [ ]:
df=pd.read_csv('/kaggle/input/road-crushes/Crash_Data_20250227.csv')
pd.set_option('display.max_columns', None)

Lets see what is the shape of our dataset

In [ ]:
print(f'The dataset has {df.shape[0]} rows and {df.shape[1]} columns')

Now,lets see 5 random rows to understand better:

In [ ]:
df.sample(5)

Lets see the columns of the dataframe:

In [ ]:
df.columns

<h3>Data columns dictionary</h3>

* **Accident Number**- Number used to identify a traffic crash report in CSPD's records management system.
* **Citation Number**- Number used to identify a citation (ticket) in CSPD's records management system.
* **Citation Issued**- Indicates whether or not a citation (ticket) was issued in the crash.
* **Accident Date Time**- Date and time of crash.
* **Unit Type**- Indicates whether at fault driver was in a vehicle or on a bicycle.
* **Vehicle Type**- Type of vehicle driven by at fault driver.
* **Age of At Fault Person**- Age of person determined to be at fault in the crash.
* **Race of At Fault Person**- Race of person determined to be at fault in the crash.
* **Gender of At Fault Person**- Gender of person determined to be at fault in the crash.
* **Ethnicity of At Fault Person**- Ethnicity of person determined to be at fault in the crash.
* **Driver Action**- Action that best describes the driver action that led to the crash. Choices are determined by State of Colorado crash report form.
* **Driver Contributing Factor**- Conditions that contributed to the crash. Choices are determined by State of Colorado crash report form.
* **First Harmful Event**- First point of injury or damage in the sequence of events in a traffic crash.
* **Zone Sector**- CSPD division and sector where crash occurred.
* **Involved Injury**- Indicates whether crash involved injury.
* **Number Injured**- Number of people injured in crash.
* **Involved Death**- Indicates whether the crash involved the death of a person.
* **Number Killed**- Number of people killed in the crash.
* **Motorcycle Involved**- Indicates whether a motorcycle was involved in the crash.
* **Motorcycle At Fault**- Indicates whether an involved motorcycle was at fault in the crash.
* **Bicycle Involved**- Indicates whether a bicycle was involved in the crash.
* **Bicycle At Fault**- Indicates whether an involved bicycle was at fault in the crash.
* **Pedestrian Involved**- Indicates whether a pedestrian was involved in the crash.
* **Pedestrian At Fault**- Indicates whether a pedestrian was at fault in the crash.
* **Crash Location**- Street address where crash occurred.
* **City**- City in which crash occurred.
* **Zip Code**- Zip code in which crash occurred.
* **Location Point**- Latitude and longitude of crash location.
* **Patrol Division**- CSPD Patrol Division in which crash occurred.



Now,lets check the data type of all columns:

In [ ]:
df.dtypes

We see that the column 'Accident Date Time' is an object lets convert it to datetime

In [ ]:
df['Accident Date Time']=pd.to_datetime(df['Accident Date Time'], format="%m/%d/%Y %I:%M:%S %p")

Lets check how much null values we have in each column:

In [ ]:
df.isna().sum()

handling with null values:
* I chose to drop unnecessary columns that i will not use in this analysis
* I chose to drop rows with null values in columns crucial to my analysis
* I fill the null values in numerical columns with mode (in all of them the mode is either 0 or 'NO Factor') so that they not impact the analysis

In [ ]:
df.dropna(subset=['Vehicle Type','Age of At Fault Person','Ethnicity of At Fault Person','Race of At Fault Person','Zone Sector'],inplace=True)
df.drop(columns=['Citation Number','Crash Location','Unit Type','Driver Action','First Harmful Event','City','Location Point','Zip Code','Patrol Division'],inplace=True)
for col in ['Number Injured', 'Number Killed', 'Driver Contributing Factor']:
    df[col].fillna(df[col].mode()[0], inplace=True)

Checking duplicated rows:

In [ ]:
df.duplicated().sum()

There are some duplicated rows, lets get rid of them:

In [ ]:
df.drop_duplicates(inplace=True)

Checking the amount of unique items of each column

In [ ]:
print(f'number of rows: {len(df)}')
df.nunique()

We see that we have duplicates in Accident Number so after checking its because its the same crash but 1 row with no citation issued and 1 row the opposite, so lets get rid of that rows

In [ ]:
df=df[~df['Accident Number'].duplicated()]

Validation: Checking that there is no contradiction in the dataframe.

In [ ]:
print(((df['Involved Death']=='Yes') & (df['Number Killed']==0)).sum())
print(((df['Involved Injury']=='Yes') & (df['Number Injured']==0)).sum())
print(((df['Motorcycle Involved']=='No') & (df['Motorcycle At Fault']=='Yes')).sum())
print(((df['Bicycle Involved']=='No') & (df['Bicycle At Fault']=='Yes')).sum())
print(((df['Pedestrian Involved']=='No') & (df['Pedestrian At Fault']=='Yes')).sum())


We see contradiction,so lets see if its really contradiction:

In [ ]:
df[(df['Bicycle Involved']=='No') & (df['Bicycle At Fault']=='Yes')]

We see that the reason for Bicycle involved being negative its because the column Vehicle Type is written as bicycle so its not contradiction.

Lets see some statistics about the numeric columns

In [ ]:
df.describe()

We can understand from the table that 'Age of At Fault Person' column is right skewed

Checking whos the gender that involve with more accidents

In [ ]:
df['Gender of At Fault Person'].value_counts().reset_index()

We see that males was at fault more than females

## <h2>Exploratory Data Analysis</h2>

Lets see which zones have the most accidents

In [ ]:
df['Zone Sector'].value_counts().reset_index().sort_values('count',ascending=False)

We can see that there are areas with a high number of accidents like 44 and 54.

In [ ]:
df_zone_nk=df.groupby('Zone Sector',as_index=False)['Number Killed'].sum().sort_values(by='Number Killed',ascending=False)
fig=px.bar(data_frame=df_zone_nk[df_zone_nk['Number Killed']>0],x='Zone Sector',y='Number Killed')
fig.update_xaxes(
    dtick=1             
)
fig.show()


We see that zones 44 and 54 have the also the most deathes in accidents, maybe in these areas, there are dangerous roads or intersections.

Lets see the distribution of the age of at fault person:

In [ ]:
px.histogram(data_frame=df,x='Age of At Fault Person',title='Age of At Fault Person distribution')

We see that young people are more involved in accidents than others, and as we guessed, the graph is right-skewed.

Checking the distribution of accidents by hours

In [ ]:
df['Hour']=df['Accident Date Time'].dt.hour
px.histogram(data_frame=df,x='Hour',title='Hour distribution')

We see that the most accidents occur during the hours when people are going to work or returning from work (7:00-8:00 and 15:00-17:00).

Checking the distribution of accidents by months

In [ ]:
df['Month']=df['Accident Date Time'].dt.month
px.histogram(data_frame=df,x='Month',title='Month distribution')

I dont see much difference between the monthes

Checking the distribution of accidents by the 'Vehicle Type' column, which represents the type of vehicle at fault.

In [ ]:
px.histogram(data_frame=df,x='Vehicle Type',title='Vehicle Type distribution')

We see that the vast majority of the accidents are caused by automobile. After automobile, its trucks.

Lets see some statistics about the accidents:

In [ ]:
print(f'{(len(df[df["Involved Death"]=="Yes"])/len(df)) * 100:.2f}% of the accidents involved death')
print(f'{(len(df[df["Involved Injury"]=="Yes"])/len(df)) * 100:.2f}% of the accidents involved injury')
print(f'{(len(df[df["Motorcycle Involved"]=="Yes"])/len(df)) * 100:.2f}% of the accidents involved Motorcycle')
print(f'{(len(df[df["Bicycle Involved"]=="Yes"])/len(df)) * 100:.2f}% of the accidents involved Bicycle')
print(f'{(len(df[df["Pedestrian Involved"]=="Yes"])/len(df)) * 100:.2f}% of the accidents involved Pedestrian')
print(f'{(len(df[df["Citation Issued"]=="Yes"])/len(df)) * 100:.2f}% of the accidents issued citation')


Adding new column 'age_at_fault_category' that maps between ages to category: 0-18 is kid, 18-36 is young, 36-55 is senior and 55 and above is old. Adding column 'severe_level' that represent the level of the result of the accident: 3 if the accident involved death, 2 if the accident involved injury and 1 otherwise. Also we adding another column 'Number_of_casualties' that represent the casualties of the accident (deaths and injuries)

In [ ]:
def compute_severe_level_accident(row):
    if row['Involved Death']=='Yes':
        return 3
    if row['Involved Injury']=='Yes':
        return 2
    return 1
def age_to_category(row):
    if row['Age of At Fault Person'] < 18:
        return 'Kid (0-18)'
    if row['Age of At Fault Person']< 36:
        return 'Young (18-36)'
    if row['Age of At Fault Person']< 55:
        return 'Senior (36-55)'
    else:
        return 'Old (55<)'
df['age_at_fault_category']=df.apply(age_to_category, axis=1)
df['severe_level']=df.apply(compute_severe_level_accident,axis=1)
severe_accidents=df[(df['Involved Death']=='Yes') | (df['Involved Injury']=='Yes')]
df['Number_of_casualties']=df['Number Injured']+df['Number Killed']

Creating histogram of age_at_fault_category:

In [ ]:
px.histogram(data_frame=df,x='age_at_fault_category',title='accidents by age category')

Visualizing the distribution of accidents by gender at fault in severe accidents:

In [ ]:
px.histogram(data_frame=severe_accidents,x='Gender of At Fault Person',title='Gender of At Fault Person in severe accidents distribution')

Checking how much each of the categories is at fault in the severe accident.

In [ ]:
gender_involved_death=severe_accidents['age_at_fault_category'].value_counts().reset_index()
gender_involved_death

Lets see it in graph:

In [ ]:
px.bar(data_frame=gender_involved_death,x='age_at_fault_category',y='count',title='Severe accidents by age category')

## <h2>Advanced Analysis</h2>

We see that young people (18-36) are the most at fault in severe accidents, while kids are the least at fault.

In [ ]:
sns.barplot(data=df, x="age_at_fault_category", y="Number_of_casualties", hue="severe_level",estimator=sum)
plt.title("Number of Casualties by Age Category and Severity Level")
plt.show()

We see that in both severe levels 1 and 2, young people (18-36) are the most at fault in severe accidents, while kids are the least at fault.

Creating pivot table: rows as Gender of At Fault Person, columns as severe_level and the values are the mean age at fault person

In [ ]:
df.pivot_table(values='Age of At Fault Person', index='Gender of At Fault Person', columns='severe_level', aggfunc='mean')

Adding a new column, 'two_wheeled_vehicle_involved', that represents whether a two-wheeled vehicle (bicycle or motorcycle) was involved in the accident. Also, creating a histogram to show how many accidents involved two-wheeled vehicles in severe accidents.

In [ ]:
def two_wheeled_vehicle_involved(row):
    if row['Vehicle Type'] == 'Bicycle':
        return 'Yes'
    if row['Vehicle Type'] == 'Motorcycle':
        return 'Yes'
    if row['Motorcycle Involved'] == 'Yes':
        return 'Yes'
    if row['Bicycle Involved'] == 'Yes':
        return 'Yes'
    else:
        return 'No'
df['two_wheeled_vehicle_involved']=df.apply(two_wheeled_vehicle_involved, axis=1)
px.histogram(data_frame=df[(df['Involved Death']=='Yes') | (df['Involved Injury']=='Yes')],x='two_wheeled_vehicle_involved',title='Two wheeled vehicle in severe accidents')


While I thought there would be more 'yes', there are more severe accidents without two-wheeled vehicles involved than with.

In [ ]:
df['Year']=df['Accident Date Time'].dt.year
px.histogram(data_frame=df[(df['Involved Death']=='Yes') | (df['Involved Injury']=='Yes')],x='Year',y='Number Killed',color='Gender of At Fault Person',title='Distribution of years seperated by gender at fault')

We can see that 2022 was the deadliest year, and males were more involved in accidents.

Creating bar plots that represent the number of casualties each year, separated by the gender of the at-fault person

In [ ]:
fig = go.Figure() 
for year in [2016,2017,2018,2019,2020,2021,2022,2023,2024,2025]: 
    df_year = df[df['Year'] == year].groupby('Gender of At Fault Person',as_index=False)['Number_of_casualties'].sum()  
    fig.add_trace(go.Bar(x=df_year['Gender of At Fault Person'],y=df_year['Number_of_casualties'], name=year))
dropdown_buttons = [ 
{'label': '2016', 'method': 'update', 
'args': [{'visible': [True, False, False, False, False, False, False, False, False, False]}, 
{'title': '2016'}]},  
{'label': '2017', 'method': 'update', 
'args': [{'visible': [False, True, False, False, False, False, False, False, False, False]}, 
{'title': '2017'}]},  
{'label': '2018', 'method': 'update', 
'args': [{'visible': [False, False, True, False, False, False, False, False, False, False]}, 
{'title': '2018'}]}, 
    {'label': '2019', 'method': 'update', 
'args': [{'visible': [False, False, False, True, False, False, False, False, False, False]}, 
{'title': '2019'}]}, 
    {'label': '2020', 'method': 'update', 
'args': [{'visible': [False, False, False, False, True, False, False, False, False, False]}, 
{'title': '2020'}]}, 
    {'label': '2021', 'method': 'update', 
'args': [{'visible': [False, False, False, False, False, True, False, False, False, False]}, 
{'title': '2021'}]}, 
    {'label': '2022', 'method': 'update', 
'args': [{'visible': [False, False, False, False, False, False, True, False, False, False]}, 
{'title': '2022'}]}, 
    {'label': '2023', 'method': 'update', 
'args': [{'visible': [False, False, False, False, False, False, False, True, False, False]}, 
{'title': '2023'}]}, 
    {'label': '2024', 'method': 'update', 
'args': [{'visible': [False, False, False, False, False, False, False, False, True, False]}, 
{'title': '2024'}]}, 
    {'label': '2025', 'method': 'update', 
'args': [{'visible': [False, False, False, False, False, False, False, False, False, True]}, 
{'title': '2025'}]}
]
fig.update_layout({ 
'updatemenus':[{ 
'type': "dropdown", 
'x': 0.99, 
'y': 0.99, 
'showactive': True, 
'active': 0, 
'buttons': dropdown_buttons}] 
}) 
fig.update_layout({'xaxis': {'title': {'text': 'Gender of At Fault Person'}}, 
'yaxis': {'title':{'text': 'Number of casualties'}}}) 
fig.show()

We see that every year, males are involved in accidents with more casualties, except for 2025, for which we have less information.

Creating scatter graph for 'Age of At Fault Person' and 'Number_of_casualties' columns.

In [ ]:
fig=px.scatter(data_frame=df,x='Age of At Fault Person',y='Number_of_casualties',color='Gender of At Fault Person',title='Scatter plot of age at fault and number of casualties',opacity=0.75)
fig.update_layout(
    yaxis=dict(range=[0, df['Number_of_casualties'].max() + 2])
)

fig.show()

We see that theres no connection between the 2 columns.

Creating a histogram of severe accidents in weather a pedestrian was involved.

In [ ]:
px.histogram(data_frame=severe_accidents,x='Pedestrian Involved',title='Distribution of Pedestrian Involved')

We can see that pedestrians were not involved in many severe accidents.

Creating box graph for the Age of At Fault Person column separated by a Gender of At Fault Person

In [ ]:
px.box(data_frame=df,y='Age of At Fault Person',color='Gender of At Fault Person',title='Box plot by gender')

We see that female, male and unknown have outliers and their median is approximately the same, male and female have larger distribution from others.

Creating box graph for the Age of At Fault Person column separated by a Race of At Fault Person

In [ ]:
px.box(data_frame=df,y='Age of At Fault Person',color='Race of At Fault Person',title='Box plot by race')

We see that white have the largest distribution, the median of asian is the biggest, we also see that white, unknown, black and indian have outliers, also black and indian have the smallest distribution. 

Creating box graph for the Age of At Fault Person column separated by a Ethnicity of At Fault Person

In [ ]:
px.box(data_frame=df,y='Age of At Fault Person',color='Ethnicity of At Fault Person',title='Box plot by Ethnicity')

We see that they all have outliers, the distribution of hispanic is the smallest, and hispanic have the smallest median.

Here we created them all at once together.

In [ ]:
fig = make_subplots(rows=1, cols=3, subplot_titles=('By Gender', 'By Race','By Ethnicity'))
fig.add_trace(go.Box(
    y=df['Age of At Fault Person'],
    x=df['Gender of At Fault Person'],
    name='By Gender',
    boxmean=True 
),row=1,col=1)
fig.add_trace(go.Box(
    y=df['Age of At Fault Person'],
    x=df['Race of At Fault Person'],
    name='By Race',
    boxmean=True 
),row=1,col=2)
fig.add_trace(go.Box(
    y=df['Age of At Fault Person'],
    x=df['Ethnicity of At Fault Person'],
    name='By Ethnicity',
    boxmean=True
),row=1,col=3)

fig.update_layout(
    title="Box Plots for Age of At Fault Person",
    showlegend=False,
    boxmode='group'
)

fig.show()

Checking the correlation between the numeric columns.

In [ ]:
df.select_dtypes('number').corr()

We see that there isnt much correlation between the variables (Except for the Number Injured to Number_of_casualties where one is inside the other)

Creating an histogram of Citation Issued column

In [ ]:
px.histogram(data_frame=df,x='Citation Issued',title='Citation Issued distribution')

We see that in most accidents, a citation was given.

In [ ]:
pd.crosstab(df['Citation Issued'],df['age_at_fault_category'],normalize='columns')

We see that in accidents where kids are at fault, a citation is given more often than in the others accidents and also senior was given less than the others.

In [ ]:
pd.crosstab(df['Citation Issued'],df['Race of At Fault Person'],normalize='columns')

We see that in accidents where black and unknown are at fault, a citation is given less often. Also asian receive the most citation.

In [ ]:
pd.crosstab(df['Citation Issued'],df['Ethnicity of At Fault Person'],normalize='columns')

We see that all of them receive approximately the same citaion.

Lets see all the graphs of numeric columns:

In [ ]:
sns.pairplot(data=df)

As we saw earlier, since there is no correlation between the variables, the graphs appear scattered.

Let's look at the combination of columns Pedestrian Involved, Motorcycle Involved, and Bicycle Involved with the number of casualties

In [ ]:
df.groupby(['Pedestrian Involved','Motorcycle Involved','Bicycle Involved'],as_index=False)['Number_of_casualties'].sum()


Create an histogram of the column 'Driver Contributing Factor'

In [ ]:
px.histogram(data_frame=df,x='Driver Contributing Factor',title='Driver Contributing Factor distribution')

We see that most of the accidents were caused by 'No apparent factor', but there are quite a few accidents caused by Driving under the influence (DUI), Driving While Ability Impaired (DWAI), and Driving while Under the Influence of Drugs (DUID),inexperience and distraction.

Lets see the number of casualties caused by each Driver Contributing Factor.

In [ ]:
driver_contributing_death=df.groupby('Driver Contributing Factor',as_index=False)['Number_of_casualties'].sum().sort_values(by='Number_of_casualties',ascending=False)
driver_contributing_death

We see that except 'no factor' or 'other factor', Driving under the influence (DUI), Driving While Ability Impaired (DWAI), and Driving while Under the Influence of Drugs (DUID),inexperience and distraction was the main factors for a large number of casualties

Visualizing 'Driver Contributing Factor' and 'Number_of_casualties'

In [ ]:
px.bar(data_frame=driver_contributing_death,x='Driver Contributing Factor',y='Number_of_casualties',title='Driver Contributing Factor by number of casualties')